Netlogo is often run from its own GUI, but it can also be run 'headless'; that is to say, run from the command line or from within another program, in the background. That is what we are doing here from this notebook. We will use the [NLRX package](https://nldoc.github.io/nlrx/) to interface between R and Netlogo. This gives us the advantage of gathering data for analysis within R while running models written in the nlogo language. We can write models using Python or other langugages, but Netlogo's language is particularly well-suited for agent modeling.

In [7]:
# this package was installed during the creation of the binder
# if you get an error running this line:
library("nlrx", lib.loc="/srv/rlibs")

# insert the following line into a block *above* this one, remove the #, and then run it:
# devtools::install_github("nldoc/nlrx")

We create a variable, `nl`, that tells the package where to find Netlogo (which we installed when we built this binder), where to find the model, written in `nlogo` code, and how much memory to set aside for the java virtual machine that will run the model.

In [8]:
nl <- nl(nlversion = "6.0.4",
         nlpath = "/home/jovyan/NetLogo 6.0.4/",
         modelpath = "/home/jovyan/NetLogo 6.0.4/app/models/Sample Models/Biology/Wolf Sheep Predation.nlogo",
         jvmmem = 1024)

Now, let's define the experiment that we are going to run, by specifying all of the various parameters available in the model.

In [9]:
nl@experiment <- experiment(expname="wolf-sheep",
                            outpath="./out/",
                            repetition=1,
                            tickmetrics="true",
                            idsetup="setup",
                            idgo="go",
                            idfinal=NA_character_,
                            idrunnum=NA_character_,
                            runtime=50,
                            evalticks=seq(40,50),
                            metrics=c("count sheep", "count wolves", "count patches with [pcolor = green]"),
                            variables = list('initial-number-sheep' = list(min=50, max=150, step=10, qfun="qunif"),
                                             'initial-number-wolves' = list(min=50, max=150, step=10, qfun="qunif")),
                            constants = list("model-version" = "\"sheep-wolves-grass\"",
                                             "grass-regrowth-time" = 30,
                                             "sheep-gain-from-food" = 4,
                                             "wolf-gain-from-food" = 20,
                                             "sheep-reproduce" = 4,
                                             "wolf-reproduce" = 5,
                                             "show-energy?" = "false"))

From the nlrx package documentation: "While the experiment defines the variables and specifications of the model, the simulation design creates a parameter input table based on these model specifications and the chosen simulation design method. nlrx provides a bunch of different simulation designs, such as full-factorial, latin-hypercube, sobol, morris and eFast. A simulation design is attached to a nl object by using one of these simdesign functions:"

In [10]:
nl@simdesign <- simdesign_lhs(nl=nl,
                               samples=100,
                               nseeds=3,
                               precision=3)

Creating latin hypercube simulation design


Now we run the simulation

In [ ]:
future::plan(multisession)

results %<-% run_nl_all(nl = nl, cleanup = "all")